In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import os
from pathlib import Path
from typing import List

import torch
import pandas as pd
import numpy as np

from hydra import initialize, compose
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf

import matplotlib.pyplot as plt

from bliss.simulator.simulated_dataset import CachedSimulatedDataset, FileDatum


In [5]:
with initialize(config_path="../", version_base=None):
    cfg = compose("config", overrides={"surveys.sdss.load_image_data=true"})

In [6]:
cfg.prior

{'_target_': 'case_studies.galaxy_clustering.prior.GalaxyClusterPrior', 'survey_bands': ['u', 'g', 'r', 'i', 'z'], 'reference_band': 2, 'star_color_model_path': '${simulator.survey.dir_path}/color_models/star_gmm_nmgy.pkl', 'gal_color_model_path': '${simulator.survey.dir_path}/color_models/gal_gmm_nmgy.pkl', 'n_tiles_h': 56, 'n_tiles_w': 56, 'tile_slen': 2, 'batch_size': 2, 'max_sources': 6, 'mean_sources': 0.48, 'min_sources': 0, 'prob_galaxy': 0.2, 'star_flux_exponent': 0.9859821185389767, 'star_flux_truncation': 5685.588160703261, 'star_flux_loc': -1.162430157551662, 'star_flux_scale': 1.4137911256506595, 'galaxy_flux_truncation': 1013, 'galaxy_flux_exponent': 0.47, 'galaxy_flux_scale': 0.6301037, 'galaxy_flux_loc': 0.0, 'galaxy_a_concentration': 0.39330758068481686, 'galaxy_a_loc': 0.8371888967872619, 'galaxy_a_scale': 4.432725319432478, 'galaxy_a_bd_ratio': 2.0}

In [7]:
cfg.cached_simulator

{'_target_': 'bliss.simulator.simulated_dataset.CachedSimulatedDataset', 'batch_size': 64, 'splits': '0:80/80:90/90:100', 'num_workers': 0, 'cached_data_path': '${generate.cached_data_path}', 'file_prefix': '${generate.file_prefix}'}

In [42]:
os.getcwd()

'/home/gapatron/bliss/case_studies/galaxy_clustering/notebooks'

In [48]:
col_names = ["RA", "DEC", "X", "Y", "MEM", "FLUX_R", "FLUX_G", "FLUX_I", "FLUX_Z", "TSIZE", "FRACDEV", "G1", "G2"]
sample_data_0 = pd.read_csv("../data/padded_catalogs/galsim_des_padded_000.dat", sep=" ", header=None, names=col_names)
sample_data_1 = pd.read_csv("../data/padded_catalogs/galsim_des_padded_001.dat", sep=" ", header=None, names=col_names)
sample_data_2 = pd.read_csv("../data/padded_catalogs/galsim_des_padded_002.dat", sep=" ", header=None, names=col_names)

In [50]:
sample_data_0.head()

,RA,DEC,X,Y,MEM,FLUX_R,FLUX_G,FLUX_I,FLUX_Z,TSIZE,FRACDEV,G1,G2
0,50.691779,-40.194891,3338.596661,3110.419714,1.0,382.122242,675.315929,590.336925,732.066957,1.363548,0,0.013826,0.010695
1,50.694299,-40.187001,3383.965626,3252.443755,1.0,182.943014,347.493597,290.063571,395.631872,2.818490,0,0.593251,0.005887
2,50.673540,-40.214869,3010.300150,2750.813487,1.0,226.558620,429.604801,362.815290,400.780268,3.689208,0,-0.192815,0.141709
3,50.690939,-40.205187,3323.487794,2925.097175,1.0,346.894552,658.902660,410.178367,516.066786,2.501445,0,0.443862,0.048449
4,50.695891,-40.203018,3412.615626,2964.123415,1.0,161.952231,540.539178,249.028774,667.967572,2.477649,0,-0.053911,-0.111394


In [46]:
len(sample_data_0)

2200

In [47]:
len(sample_data_1)

2200

In [49]:
len(sample_data_2)

2200

In [72]:
d = dict()
d["plocs"] = torch.tensor([sample_data_0[["X", "Y"]].to_numpy(), sample_data_1[["X", "Y"]].to_numpy()])
n_sources = torch.sum(d["plocs"][:,:,0] != 0, axis=1)
d["n_sources"] = n_sources

In [70]:
n_sources

tensor([2112, 2026])

In [26]:
from bliss.catalog import FullCatalog, TileCatalog

In [68]:
torch.sum(n_sources[:,:,0] != 0, axis=1)

tensor([2112, 2026])

In [65]:
n_sources[:,:,0] != 0

tensor([[ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False]])

In [73]:
fc = FullCatalog(height=5000, width=5000, d=d)

In [75]:
fc.batch_size

2

In [77]:
tc = fc.to_tile_catalog(20, 20)

In [82]:
tc.width

5000

In [84]:
tile_catalog_dict = tc.to_dict()

In [85]:
tile_catalog_dict.keys()

dict_keys(['locs', 'n_sources'])

In [91]:
tile_catalog_dict["n_sources"][0].sum()

tensor(2112)

In [99]:
tile_catalog_dict["n_sources"][0]

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [100]:
tile_catalog_dict["locs"][0][1][1]

tensor([[0.5930, 0.4674],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000]])